# 🎨 Chroma Vector Store in LangChain

## 📚 What is Chroma?

**Chroma** is an AI-native open-source vector database focused on **developer productivity and happiness**. It's designed to make building AI applications simple and intuitive.

### 🎯 Key Features:
- 🚀 **Easy to Use**: Simple API, great developer experience
- 🆓 **Open Source**: Apache 2.0 license
- 🔌 **Embedded Mode**: Runs in-process (no server needed!)
- 🌐 **Client-Server Mode**: Scales for production
- 🏷️ **Metadata Filtering**: Rich filtering capabilities
- 🔄 **Automatic Persistence**: Built-in save/load

### 🧠 How Chroma Works:

```
┌─────────────┐    ┌──────────────┐    ┌─────────────────┐
│  Documents  │───▶│  Embeddings  │───▶│ Chroma Collection│
│  + Metadata │    │   + IDs      │    │   (SQLite DB)    │
└─────────────┘    └──────────────┘    └─────────────────┘
                                              │
                                              ▼
┌─────────────┐    ┌──────────────┐    ┌─────────────────┐
│   Results   │◀───│  Similarity  │◀───│     Query       │
│  + Scores   │    │   Search     │    │  + Filters      │
└─────────────┘    └──────────────┘    └─────────────────┘
```

### 🔗 Resources:
- 📖 [LangChain Vector Stores](https://python.langchain.com/v0.2/docs/integrations/vectorstores/)
- 🎨 [Chroma Documentation](https://docs.trychroma.com/)

---
## 📦 Step 1: Setup and Installation

```bash
pip install chromadb langchain langchain-chroma
```

## 📄 Step 2: Import Libraries and Load Documents

### Understanding the Components:

| Component | Purpose |
|-----------|---------|
| `Chroma` | The vector store class |
| `TextLoader` | Loads plain text files |
| `OllamaEmbeddings` | Generates embeddings locally |
| `RecursiveCharacterTextSplitter` | Smartly splits text preserving structure |

### 🔄 RecursiveCharacterTextSplitter vs CharacterTextSplitter:

```
CharacterTextSplitter:      RecursiveCharacterTextSplitter:
├─ Simple splitting         ├─ Intelligent splitting
├─ Fixed separator          ├─ Multiple separators
└─ May break sentences      └─ Preserves structure
                               ├─ First: \n\n (paragraphs)
                               ├─ Then: \n (lines)
                               ├─ Then: " " (words)
                               └─ Finally: "" (characters)
```

In [ ]:
# 📥 Import required libraries
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

print("✅ All libraries imported successfully!")

In [ ]:
# 📄 Load the document
loader = TextLoader("speech.txt")
data = loader.load()

print(f"📊 Loaded {len(data)} document(s)")
print(f"📏 Total characters: {len(data[0].page_content)}")
print(f"📋 Metadata: {data[0].metadata}")
print("\n📄 Preview (first 200 chars):")
print(data[0].page_content[:200] + "...")

[Document(page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in

## ✂️ Step 3: Split Documents into Chunks

### Why Smaller Chunk Sizes (500)?

For Chroma, we're using smaller chunks compared to FAISS:
- **More precise retrieval**: Smaller chunks = more targeted results
- **Better for Q&A**: Reduces noise in retrieved context
- **Trade-off**: More chunks to search through

### Parameters Explained:
- `chunk_size=500`: Each chunk max 500 characters
- `chunk_overlap=0`: No overlap (can add for context continuity)

In [ ]:
# ✂️ Split the document into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,     # Smaller chunks for more precise retrieval
    chunk_overlap=0     # No overlap between chunks
)
splits = text_splitter.split_documents(data)

print(f"📦 Created {len(splits)} chunks from the document\n")

# 👀 Examine the chunks
for i, split in enumerate(splits):
    print(f"📄 Chunk {i+1}: {len(split.page_content)} characters")
print(f"\n📊 Average chunk size: {sum(len(s.page_content) for s in splits) / len(splits):.0f} characters")

## 🏗️ Step 4: Create Chroma Vector Store

### In-Memory vs Persistent Mode:

| Mode | Use Case | Persistence |
|------|----------|-------------|
| **In-Memory** | Quick experiments, testing | ❌ Lost on restart |
| **Persistent** | Production, large datasets | ✅ Saved to disk |

### Creating the Vector Store:
```python
# In-Memory (ephemeral)
vectordb = Chroma.from_documents(documents=splits, embedding=embedding)

# Persistent (saved to disk)
vectordb = Chroma.from_documents(documents=splits, embedding=embedding, persist_directory="./chroma_db")
```

In [ ]:
# 🧬 Initialize embedding model (using Ollama - runs locally)
embedding = OllamaEmbeddings()

# 🏗️ Create Chroma vector store (in-memory mode)
# This will:
# 1. Generate embeddings for each document chunk
# 2. Store them in an in-memory Chroma collection
vectordb = Chroma.from_documents(documents=splits, embedding=embedding)

print("✅ Chroma vector store created successfully!")
print(f"📊 Collection contains {vectordb._collection.count()} documents")

## 🔍 Step 5: Similarity Search

### Query the Vector Store

Now we can ask questions and find the most relevant document chunks!

### How it Works:
```
Your Question ──▶ Embedding ──▶ Compare with all vectors ──▶ Top K results
```

In [ ]:
# 🔍 Perform similarity search
query = "What does the speaker believe is the main reason the United States should enter the war?"

# Search for most similar documents (default: returns top 4)
docs = vectordb.similarity_search(query)

print(f"🔍 Query: {query}")
print(f"📊 Found {len(docs)} similar documents\n")
print("=" * 60)
print("📄 Most Relevant Result:")
print("=" * 60)
print(docs[0].page_content)

'To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'

## 💾 Step 6: Persistent Storage

### Why Persist Your Vector Store? 🤔

**Benefits:**
- ⚡ **Faster startup**: No need to re-embed documents
- 💰 **Cost savings**: Don't pay for embeddings again
- 🔄 **Reusability**: Share across sessions/applications

### Chroma Storage Format:
```
chroma_db/
├── chroma.sqlite3           ← SQLite database (metadata + vectors)
└── {uuid}/                  ← Collection data
    ├── data_level0.bin      ← Vector data
    ├── header.bin           ← Index header
    ├── index_metadata.json  ← Index configuration
    └── length.bin           ← Length data
```

In [ ]:
# 💾 Create a persistent Chroma vector store
# This creates a new collection AND saves it to disk
vectordb_persistent = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory="./chroma_db"  # 📁 Data will be saved here
)

print("✅ Persistent Chroma DB created!")
print("📁 Location: ./chroma_db/")

# 📂 Check what was saved
import os
if os.path.exists("./chroma_db"):
    print(f"📄 Contents: {os.listdir('./chroma_db')}")

## 📂 Step 7: Loading from Disk

### Load an Existing Collection

When you restart your application, you can load the existing Chroma database:

```python
db = Chroma(
    persist_directory="./chroma_db",
    embedding_function=embedding
)
```

**Note:** You need to provide the same embedding function used during creation!

In [ ]:
# 📂 Load existing Chroma DB from disk
db2 = Chroma(
    persist_directory="./chroma_db",
    embedding_function=embedding
)

# 🔍 Test the loaded database
docs = db2.similarity_search(query)

print("✅ Chroma DB loaded successfully!")
print(f"📊 Collection size: {db2._collection.count()} documents")
print("\n" + "=" * 60)
print("📄 Search Result from Loaded DB:")
print("=" * 60)
print(docs[0].page_content)

To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.


## 📊 Step 8: Similarity Search with Scores

### Understanding Chroma Scores 🎯

Chroma returns a **distance score** (like FAISS):
- **Lower score** = More similar (better match)
- The exact distance metric depends on configuration (default: L2)

### Why Use Scores?
- 🎚️ **Set thresholds**: Filter out low-confidence results
- 📈 **Ranking**: Compare relevance across results
- 🔍 **Debugging**: Understand why certain results were returned

In [ ]:
# 📊 Similarity search with scores
docs_with_scores = vectordb.similarity_search_with_score(query)

print(f"🔍 Query: {query}\n")
print("=" * 60)
print("📊 Results with Distance Scores")
print("=" * 60)

for i, (doc, score) in enumerate(docs_with_scores):
    print(f"\n📄 Result {i+1}")
    print(f"   📏 Distance Score: {score:.4f}")
    confidence = "🟢 High" if score < 1.0 else "🟡 Medium" if score < 1.5 else "🔴 Low"
    print(f"   🎯 Confidence: {confidence}")
    print(f"   📝 Preview: {doc.page_content[:100]}...")

[(Document(page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.', metadata={'source': 'speech.txt'}),
  15573.843876438417),
 (Document(page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.', metadata={'source': 'speech.txt'}),
  18241.307592374058),
 

## 🔄 Step 9: Using as a Retriever

### What is a Retriever?

A **Retriever** is a LangChain interface that standardizes how we fetch documents. This allows seamless integration with:

- 🔗 **Chains**: Complex multi-step workflows
- 🤖 **Agents**: Autonomous AI systems
- 📝 **RAG**: Retrieval Augmented Generation

### Retriever Configuration Options:
```python
retriever = vectordb.as_retriever(
    search_type="similarity",        # or "mmr" for diversity
    search_kwargs={"k": 4}           # Number of results
)
```

In [ ]:
# 🔄 Convert Chroma to a retriever
retriever = vectordb.as_retriever(
    search_type="similarity",  # Use standard similarity search
    search_kwargs={"k": 3}     # Return top 3 results
)

# 🎯 Use the retriever's invoke method
retrieved_docs = retriever.invoke(query)

print(f"🔍 Query: {query}")
print(f"📊 Retrieved {len(retrieved_docs)} documents\n")
print("=" * 60)
print("📄 Top Result:")
print("=" * 60)
print(retrieved_docs[0].page_content)

'To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'

## 🎯 Summary: Chroma in LangChain

### What We Learned:

| Concept | Description |
|---------|-------------|
| 📄 Document Loading | Load files with `TextLoader` |
| ✂️ Smart Splitting | Use `RecursiveCharacterTextSplitter` for better chunks |
| 🧬 Embeddings | Generate vectors with `OllamaEmbeddings` |
| 🏗️ Index Creation | Create Chroma collection with `from_documents()` |
| 🔍 Similarity Search | Find documents with `similarity_search()` |
| 📊 Scores | Get confidence with `similarity_search_with_score()` |
| 💾 Persistence | Save/load with `persist_directory` |
| 🔄 Retriever | Convert with `as_retriever()` |

### 🆚 FAISS vs Chroma Comparison:

| Feature | FAISS | Chroma |
|---------|-------|--------|
| **Speed** | ⚡ Faster for large datasets | 🚗 Good for moderate sizes |
| **Setup** | 📦 Requires installation | 🎯 Simpler setup |
| **Persistence** | 📁 Manual save/load | ✅ Built-in persistence |
| **Metadata** | ⚠️ Limited | ✅ Rich filtering |
| **Scalability** | 🚀 Billions of vectors | 🔄 Client-server mode |
| **Use Case** | Production search | Development & prototyping |

### 🚀 Key Takeaways:

1. **Chroma is developer-friendly** - Great for rapid prototyping
2. **Built-in persistence** - Just specify `persist_directory`
3. **Smaller chunks** - Often better for precise retrieval
4. **Retriever interface** - Essential for RAG applications

### 📚 Next Steps:
- 🔍 Try metadata filtering
- 🎛️ Experiment with MMR (Maximal Marginal Relevance)
- 🤖 Build a RAG chatbot using the retriever
- 📊 Compare with other vector stores